# 装饰器

装饰器（Decorator）本质上是高阶函数，其返回值是函数对象，  
它可以让其他函数对象在其内部实现不变动的前提下增加新的功能。  
有了装饰器，就可以抽离出许多与函数本身功能无关的代码供复用。

In [ ]:
# 先看一个简单的函数
def hello():
    print('hello world')

In [ ]:
# 如果想让hello()再实现个其他功能，比如打印当前时间，可以直接在函数体加上相应的代码
import datetime
def hello():
    print('当前时间'，datetime.datetime.now())
    print('hello world')

In [1]:
# 假设大量函数都要增加这个需求，显然不能一个个去改
# 为了减少重复代码，可以定义一个专门打印当前时间的函数，打印完时间后再执行真正的业务代码
import datetime
def print_time(func):
    print('当前时间:', datetime.datetime.now())
    func()
def hello():
    print('hello world')
print_time(hello)

当前时间: 2022-04-01 19:00:21.931267
hello world


这样修改后的代码，每次都要传递一个函数给print_time()，这种做法破坏了原有的代码逻辑结构。  
原先的业务逻辑中只需调用hello()，而现在必须要调用print_time()，因此装饰器是更好的选择：

In [2]:
# @符号是装饰器的语法糖，在定义函数时使用，可以避免再一次赋值的操作
def my_decorator(func):
    def wrapper():
        print('当前时间:', datetime.datetime.now())
        func()
    return wrapper
@my_decorator
def hello():
    print('hello world')  
hello()

当前时间: 2022-04-01 19:00:25.215788
hello world


`@my_decorator`相当于`hello = my_decorator(hello) `，运行函数hello()时的调用过程是：  
先执行`hello = my_decorator(hello)`，此时变量hello指向的是装饰器my_decorator()。  
my_decorator(func)中传参是hello，返回的wrapper又会调用到原函数hello()。  
所以先执行wrapper()里的代码，再才执行hello()里的代码。  

In [3]:
# 对于带有参数的函数，直接在对应装饰器的wrapper()上也加上对应参数就可以了
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('当前时间:', datetime.datetime.now())
        func(*args, **kwargs)
    return wrapper
@my_decorator
def hello(name):
    print('你好，{}'.format(name))
hello('Aiobisio')

当前时间: 2022-04-01 19:00:28.706392
你好，Aiobisio


In [4]:
# 打印一下上例中hello()的元信息
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('当前时间:', datetime.datetime.now())
        func(*args, **kwargs)
    return wrapper
@my_decorator
def hello(name):
    print('你好，{}'.format(name))
# hello('Aiobisio')
print(hello.__name__)  

wrapper


显而易见，以前那个hello()被wrapper()取代了，函数名等函数属性发生了改变，  
考虑到未来可能会用到元函数信息，可以用内置装饰器`@functools.wrap`来保留它。

In [5]:
import functools
def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('当前时间:', datetime.datetime.now())
        func(*args, **kwargs)
    return wrapper
@my_decorator
def hello(name):
    print('你好，{}'.format(name))
# hello('Aiobisio')
print(hello.__name__)

hello


In [6]:
# 前文所述的装饰器都是接收外来的参数，其实装饰器有自己的参数
# 下例是三层嵌套的函数，最外层my_decorator_outer()的参数num控制的是次数
def my_decorator_outer(num):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            print('当前时间:', datetime.datetime.now())
            for i in range(num):
                func(*args, **kwargs)
        return wrapper
    return my_decorator
@my_decorator_outer(3)
def hello(name):
    print('你好，{}'.format(name))
hello('Aiobisio')

当前时间: 2022-04-01 19:00:35.444263
你好，Aiobisio
你好，Aiobisio
你好，Aiobisio


除了函数，类也能实现装饰器的功能，这是由于调用一个类的实例时，会调用到它的`__call__()`方法。

In [7]:
# 除了函数，类也能实现装饰器的功能，这是由于调用一个类的实例时，会调用它的`__call__()`方法
class MyDecorator:
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print('当前时间:', datetime.datetime.now())
        return self.func(*args, **kwargs)
@MyDecorator
def hello(name):
    print('你好，{}'.format(name))
hello('Aiobisio')

当前时间: 2022-04-01 19:00:38.337068
你好，Aiobisio


In [8]:
# 如果有多个装饰器，把要用的装饰器都加上去就好，执行顺序为从上到下依次执行
import functools
def my_decorator1(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('装饰器1，当前时间:', datetime.datetime.now())
        func(*args, **kwargs)
    return wrapper
def my_decorator2(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('装饰器2，当前时间:', datetime.datetime.now())
        func(*args, **kwargs)
    return wrapper
def my_decorator3(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('装饰器3，当前时间:', datetime.datetime.now())
        func(*args, **kwargs)
    return wrapper
@my_decorator1
@my_decorator2
@my_decorator2
def hello(name):
    print('你好，{}'.format(name))
hello('Aiobisio')

装饰器1，当前时间: 2022-04-01 19:00:41.111862
装饰器2，当前时间: 2022-04-01 19:00:41.111862
装饰器2，当前时间: 2022-04-01 19:00:41.111862
你好，Aiobisio
